# Calculating Innovation Measures

This notebook calculates the atypicality score and technological leap score for each patent in the `final_fwdcitations.csv` dataset. 

## Results note

There are **763,040** patents without any any cited CPC subclasses or groups.

## Atypicality Score

This notebook calculates the Atypicality Score for all patents in the `final_fwdcitations.csv` dataset.

### Steps
1. Find the set of cpc subclasses for each patent using the `g_cpc_current.csv` dataset.

2. Filter the set of cpc subclasses to only include those that occur in the `final_fwdcitations.csv` dataset.

3. For each patent, generate all 2-way combinations of cpc subclasses. Any patent that cites fewer than 2 cpc subclasses is considered to have no 2-way combinations.

4. Count the number of occurences of each 2-way combination across all patents.

5. Calculate the Atypicality Score for each cpc-subclass-pair:
$$
\text{CPC Subclass Pair Atypicality Score} = -ln\left(\frac{n_{pair}}{N}\right) \\
\text{ where } n_{pair} \text{ is the number of occurrences of the pair and } \\
N \text{ is the sum of all occurrences of pairs.}
$$

6. Calculate the Atypicality Score for each patent:
$$
\text{Patent Atypicality Score} = \begin{cases}
\frac{\sum_{pair} \text{CPC Subclass Pair Atypicality Score}}{\text{Number of CPC Subclass Pairs cited by Patent}} & \text{if Number of CPC Subclass Pairs} > 0 \\
NaN & \text{otherwise}
\end{cases}
$$



In [2]:
import pandas as pd

In [3]:
dtypes = {
    "patent_id": "string",
    "forward_citations": "int64",
}
fwd_citations = pd.read_csv("./data/final_fwdcitation.csv", dtype=dtypes)

fwd_citations.head()

,patent_id,patent_type,patent_date,patent_title,wipo_kind,forward_citations
0,10000000,utility,2018-06-19,Coherent LADAR using intra-pixel quadrature de...,B2,13
1,10000001,utility,2018-06-19,Injection molding machine and mold thickness c...,B2,0
2,10000002,utility,2018-06-19,Method for manufacturing polymer film and co-e...,B2,0
3,10000003,utility,2018-06-19,Method for producing a container from a thermo...,B2,2
4,10000004,utility,2018-06-19,"Process of obtaining a double-oriented film, c...",B2,0


In [4]:
dtypes = {
    "patent_id": "string",
    "cpc_sequence": "int64",
}
cpc = pd.read_csv(
    "./data/g_cpc_current.tsv", 
    sep="\t",
    dtype=dtypes,
    )
cpc.head()

,patent_id,cpc_sequence,cpc_section,cpc_class,cpc_subclass,cpc_group,cpc_type
0,3950000,0,A,A63,A63C,A63C9/001,inventional
1,3950000,1,A,A63,A63C,A63C9/00,inventional
2,3950000,2,A,A63,A63C,A63C9/002,inventional
3,3950000,3,A,A63,A63C,A63C9/081,inventional
4,3950001,0,A,A63,A63C,A63C9/086,inventional


In [5]:
citations_dtypes = {
    "patent_id": "string",
    "citation_patent_id": "string",
}
citations = pd.read_csv(
    "./data/g_us_patent_citation.tsv", 
    sep="\t",
    dtype=citations_dtypes,
    )
citations.head()

/var/folders/zx/x6hb1_p95p33hh3235qrnlqm0000gn/T/ipykernel_13257/3262492686.py:5: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  citations = pd.read_csv(


,patent_id,citation_sequence,citation_patent_id,citation_date,record_name,wipo_kind,citation_category
0,10000000,0,5093563,1992-03-01,Small,A,cited by examiner
1,10000000,1,5751830,1998-05-01,Hutchinson,A,cited by applicant
2,10000001,0,7804268,2010-09-01,Park,B2,cited by examiner
3,10000001,1,9022767,2015-05-01,Oono,B2,cited by examiner
4,10000001,2,9090016,2015-07-01,Takeuchi,B2,cited by examiner


In [6]:
fwd_citations = fwd_citations[["patent_id", "forward_citations"]]
cpc = cpc[["patent_id", "cpc_sequence", "cpc_subclass", "cpc_group"]]
citations = citations[["patent_id", "citation_sequence", "citation_patent_id"]]

In [7]:
num_patents = fwd_citations["patent_id"].nunique()
print(f"Number of patents: {num_patents}")

Number of patents: 7507819


In [8]:
from itertools import combinations
import os

#1. Group by patent_id and collect all cpc_subclasses for each patent
# This will create a dataframe where each patent_id maps to a set of its cpc_subclasses
cpc_subclasses = cpc.groupby("patent_id")["cpc_subclass"].apply(set)
cpc_subclasses = pd.merge(
    cpc_subclasses,
    fwd_citations.set_index("patent_id"), 
    left_index=True, 
    right_index=True,
    how="right",
    validate="1:1"
)
cpc_subclasses = cpc_subclasses["cpc_subclass"]
cpc_subclasses = cpc_subclasses.rename("cpc_subclasses")
cpc_subclasses.head()


patent_id
10000000                                  {G01S}
10000001                            {G05B, B29C}
10000002    {B32B, B29L, B29K, B60C, B29C, B29D}
10000003                {B29K, B29L, B29D, B29C}
10000004                      {B29K, B29L, B29C}
Name: cpc_subclasses, dtype: object

In [9]:
# Check the number of patents with no cpc subclasses cited
cpc_subclasses.isna().sum()

763040

In [12]:
import ast
import numpy as np
# 2. Generate all unique pairs of cpc_subclasses for each patent
def generate_pairs(cpc_subclass: set) -> list:
    if pd.isna(cpc_subclass):
        return []
    if not isinstance(cpc_subclass, set):
        cpc_subclass = set(cpc_subclass)
    if not cpc_subclass:
        return []
    return list(combinations(sorted(cpc_subclass), 2))

patent_cpc_subclass_pairs = None
if not os.path.exists("./data/cpc_subclass_pairs.csv"):
    patent_cpc_subclass_pairs = cpc_subclasses.apply(generate_pairs)
    patent_cpc_subclass_pairs = patent_cpc_subclass_pairs.rename("cpc_subclass_pairs")
    patent_cpc_subclass_pairs.to_csv("./data/cpc_subclass_pairs.csv")
else:
    patent_cpc_subclass_pairs = pd.read_csv("./data/cpc_subclass_pairs.csv", index_col=0, dtype={"patent_id": "string"})
    patent_cpc_subclass_pairs["cpc_subclass_pairs"] = patent_cpc_subclass_pairs["cpc_subclass_pairs"].apply(ast.literal_eval)
    patent_cpc_subclass_pairs = patent_cpc_subclass_pairs["cpc_subclass_pairs"]
patent_cpc_subclass_pairs.head()

patent_id
10000000                                                   []
10000001                                       [(B29C, G05B)]
10000002    [(B29C, B29D), (B29C, B29K), (B29C, B29L), (B2...
10000003    [(B29C, B29D), (B29C, B29K), (B29C, B29L), (B2...
10000004           [(B29C, B29K), (B29C, B29L), (B29K, B29L)]
Name: cpc_subclass_pairs, dtype: object

In [13]:
# 3. Flatten the list of pairs into a long-form DataFrame
# Each row maps a patent to one of its cpc subclass pair, or NaN if it has fewer than 2 cpc subclasses cited
patent_cpc_subclass_pairs_long = patent_cpc_subclass_pairs.explode()
patent_cpc_subclass_pairs_long = patent_cpc_subclass_pairs_long.rename("cpc_subclass_pair")
patent_cpc_subclass_pairs_long = patent_cpc_subclass_pairs_long.to_frame().reset_index()
patent_cpc_subclass_pairs_long.head()

,patent_id,cpc_subclass_pair
0,10000000,NaN
1,10000001,"(B29C, G05B)"
2,10000002,"(B29C, B29D)"
3,10000002,"(B29C, B29K)"
4,10000002,"(B29C, B29L)"


In [14]:
# 4. Count occurrences of each unique pair
if not os.path.exists("./data/cpc_subclass_pair_counts.csv"):
    pair_counts = patent_cpc_subclass_pairs_long.groupby("cpc_subclass_pair").size()
    pair_counts = pair_counts.to_frame(name="count")
    pair_counts.to_csv("./data/cpc_subclass_pair_counts.csv")
else:
    pair_counts = pd.read_csv("./data/cpc_subclass_pair_counts.csv")
    pair_counts["cpc_subclass_pair"] = pair_counts["cpc_subclass_pair"].apply(ast.literal_eval)
    pair_counts = pair_counts.set_index("cpc_subclass_pair")

pair_counts.head()

,count
cpc_subclass_pair,
"(A01B, A01C)",1662
"(A01B, A01D)",1209
"(A01B, A01F)",133
"(A01B, A01G)",468
"(A01B, A01H)",15


In [15]:
total_pairs = pair_counts.sum()
print(f"Total number of pairs in the dataset: {total_pairs}")

num_unique_pairs = pair_counts.index.nunique()
print(f"Number of unique pairs in the dataset: {num_unique_pairs}")

Total number of pairs in the dataset: count    14580344
dtype: int64
Number of unique pairs in the dataset: 95614


In [16]:
import numpy as np
# 5. Assign each cpc subclass pair an atypicality score
def atypicality_score(n):
    if n > 0:
        return -np.log(n/total_pairs)
    else:
        # This should not happen since the pairs are generated from existing data. If it does, raise an error.
        raise ValueError("Count must be greater than 0")

if not os.path.exists("./data/cpc_subclass_pair_counts_with_atypicality.csv"):
    pair_counts["atypicality_score"] = pair_counts["count"].apply(atypicality_score)
    pair_counts.to_csv("./data/cpc_subclass_pair_counts_with_atypicality.csv")
else: 
    pair_counts = pd.read_csv("./data/cpc_subclass_pair_counts_with_atypicality.csv")
    pair_counts["cpc_subclass_pair"] = pair_counts["cpc_subclass_pair"].apply(ast.literal_eval)
    pair_counts = pair_counts.set_index("cpc_subclass_pair")
pair_counts.head()




,count,atypicality_score
cpc_subclass_pair,,
"(A01B, A01C)",1662,9.079408
"(A01B, A01D)",1209,9.397636
"(A01B, A01F)",133,11.604836
"(A01B, A01G)",468,10.346717
"(A01B, A01H)",15,13.787135


In [17]:
# Atypicality score per cpc subclass pair summary statistics
pair_counts["atypicality_score"].describe()

count    95614.000000
mean        14.105618
std          1.963772
min          4.489449
25%         12.857599
50%         14.415743
75%         15.802038
max         16.495185
Name: atypicality_score, dtype: float64

In [18]:
pair_counts_dict = pair_counts.to_dict(orient="index")

In [20]:
# Step 6: Calculate the atypicality score for each patent
def calculate_atypicality_score(cpc_subclass_pairs):
    if len(cpc_subclass_pairs) == 0:
        return np.nan
    # Get the atypicality score for each cpc_subclass pair
    scores = [pair_counts_dict.get(tuple(pair)).get("atypicality_score") for pair in cpc_subclass_pairs]
    if len(scores) == 0:
        raise RuntimeError("No matching pairs found for the given cpc_group_pairs.")
    
    # Calculate the atypicality score for each patent = average atypicality score for all cpc subclass pairs it cites
    atypicality_score = np.mean(scores)
    
    return atypicality_score

if not os.path.exists("./data/patents_with_atypicality.csv"):
    if not isinstance(patent_cpc_subclass_pairs, pd.DataFrame):
        patent_cpc_subclass_pairs = patent_cpc_subclass_pairs.to_frame()
    # Group by patent_id and calculate the atypicality score for each patent
    patent_cpc_subclass_pairs["atypicality_score"] = patent_cpc_subclass_pairs["cpc_subclass_pairs"].apply(calculate_atypicality_score)
    patent_cpc_subclass_pairs.to_csv("./data/patents_with_atypicality.csv")
else:
    patent_cpc_subclass_pairs = pd.read_csv("./data/patents_with_atypicality.csv", index_col=0)
    patent_cpc_subclass_pairs["cpc_subclass_pairs"] = patent_cpc_subclass_pairs["cpc_subclass_pairs"].apply(ast.literal_eval)


patent_cpc_subclass_pairs.head()

,cpc_subclass_pairs,atypicality_score
patent_id,,
10000000,[],NaN
10000001,"[(B29C, G05B)]",8.984207
10000002,"[(B29C, B29D), (B29C, B29K), (B29C, B29L), (B2...",8.375340
10000003,"[(B29C, B29D), (B29C, B29K), (B29C, B29L), (B2...",6.990229
10000004,"[(B29C, B29K), (B29C, B29L), (B29K, B29L)]",6.239670


In [21]:
# Atypicality score summary statistics
patent_cpc_subclass_pairs["atypicality_score"].describe()

count    4.056170e+06
mean     8.093600e+00
std      1.987437e+00
min      4.489449e+00
25%      6.627396e+00
50%      8.040293e+00
75%      9.427694e+00
max      1.649518e+01
Name: atypicality_score, dtype: float64

In [22]:
# 8. Merge the atypicality scores with the list of cpc subclasses to get the full output
patent_cpc_subclass_pairs_with_atypicality = pd.merge(patent_cpc_subclass_pairs, cpc_subclasses, right_index=True, left_index=True, how="inner", validate="1:1")
patent_cpc_subclass_pairs_with_atypicality.head()

,cpc_subclass_pairs,atypicality_score,cpc_subclasses
patent_id,,,
10000000,[],NaN,{G01S}
10000001,"[(B29C, G05B)]",8.984207,"{G05B, B29C}"
10000002,"[(B29C, B29D), (B29C, B29K), (B29C, B29L), (B2...",8.375340,"{B32B, B29L, B29K, B60C, B29C, B29D}"
10000003,"[(B29C, B29D), (B29C, B29K), (B29C, B29L), (B2...",6.990229,"{B29K, B29L, B29D, B29C}"
10000004,"[(B29C, B29K), (B29C, B29L), (B29K, B29L)]",6.239670,"{B29K, B29L, B29C}"


In [23]:
# Step 9: Save the final DataFrame to a CSV file
patent_cpc_subclass_pairs_with_atypicality.to_csv("./data/patents_with_atypicality.csv")

In [24]:
# 10. Validate that the output dataset contains the expected number of patents
num_patents = patent_cpc_subclass_pairs_with_atypicality.shape[0]
print(f"Number of patents in the output dataset: {num_patents}")

Number of patents in the output dataset: 7507819


In [25]:
patent_cpc_subclass_pairs_with_atypicality.isna().sum()

cpc_subclass_pairs          0
atypicality_score     3451649
cpc_subclasses         763040
dtype: int64

## Atypicality Score Summary Statistics

| Atypicality Score Statistic | Value     |
|----------------------------|-----------|
| Number of non-null values  | 4,056,170 |
| Number of null values      | 3,451,649 |
| Total Count                | 7,507,819 |
| Mean                       | 8.0936    |
| Standard Deviation         | 1.9874    |
| Min                        | 4.4894    |
| Max                        | 16.4952   |
| 25th-percentile            | 6.6274    |
| 50th-percentile            | 8.0403    |
| 75th-percentile            | 9.4277    |

In [26]:
# 11. Validate that all the patents with NaN atypicality scores are indeed patents with fewer than 2 cpc_subclass_pairs
assert patent_cpc_subclass_pairs_with_atypicality[patent_cpc_subclass_pairs_with_atypicality["atypicality_score"].isna()]["cpc_subclass_pairs"].apply(lambda x: len(x) < 2).all(), "There are patents with NaN atypicality scores that have more than 1 cpc_subclass_pairs."
print("All patents with NaN atypicality scores have fewer than 2 cpc_subclass_pairs.")

All patents with NaN atypicality scores have fewer than 2 cpc_subclass_pairs.


In [27]:
# 12. Validate that no cpc_subclass_pairs have been over-counted, i.e. the sum of counts in pair_counts matches the total number of pairs in patent_cpc_subclass_pairs_long
assert pair_counts["count"].sum() == patent_cpc_subclass_pairs_long.dropna(subset=["cpc_subclass_pair"]).shape[0], f"The sum of counts in pair_counts ({pair_counts['count'].sum()}) does not match the total number of pairs in patent_cpc_subclass_pairs_long ({patent_cpc_subclass_pairs_long.dropna(subset=['cpc_subclass_pair']).shape[0]})."
print("All cpc_subclass_pairs have been counted correctly.")

All cpc_subclass_pairs have been counted correctly.


In [31]:
# 13. Random spot check: Validate that the atypicality score for a random patent matches the expected value
import random
def manual_check(patent_id):
    cpc_subclass_pairs = patent_cpc_subclass_pairs.loc[patent_id, "cpc_subclass_pairs"]
    if isinstance(cpc_subclass_pairs, str):
        cpc_subclass_pairs = ast.literal_eval(cpc_subclass_pairs)
    if len(cpc_subclass_pairs) == 0:
        return np.nan
    counts = [pair_counts_dict.get(tuple(pair)).get("count") for pair in cpc_subclass_pairs]
    if len(counts) == 0 or None in counts:
        raise RuntimeError("No matching pairs found for the given cpc_subclass_pairs.")
    scores = [-np.log(count/total_pairs) if count > 0 else np.nan for count in counts]
    atypicality_score = np.mean(scores)
    return atypicality_score

samples = random.sample(patent_cpc_subclass_pairs_with_atypicality.index.tolist(), 10000)
for sample in samples:
    expected_score = manual_check(sample)
    actual_score = patent_cpc_subclass_pairs_with_atypicality.loc[sample, "atypicality_score"]
    assert np.isclose(expected_score, actual_score, equal_nan=True), f"Expected {expected_score}, but got {actual_score} for patent {sample}."
print("All manual checks passed successfully.")


All manual checks passed successfully.


In [32]:
num_patents_in_output = patent_cpc_subclass_pairs_with_atypicality.shape[0]
assert num_patents_in_output == num_patents, f"Number of patents in the output dataset ({num_patents_in_output}) does not match the number of patents in the input dataset ({num_patents})."
print("Number of patents in the output dataset matches the number of patents in the input dataset.")

Number of patents in the output dataset matches the number of patents in the input dataset.


## Technological Leap Score

This notebook calculates the Technological Leap Score for all patents in the `final_fwdcitations.csv` dataset.

### Steps

1. Find the set of backward citations (set of patents cited by a patent) for each patent in `g_us_patent_citations.csv` dataset.

2. Filter the set of backward citations to only include those that occur in the `final_fwdcitations.csv` dataset.

3. For each patent, find the set of CPC groups that are cited by the patent.

4. For each patent, find the set of CPC groups that are cited by its backward citations. 

5. Find the intersection of (3) and (4) to get the overlapping CPC groups.

6. Find the union of (3) and (4) to get the total CPC groups.

7. Calculate the Jaccard Similarity score = 
$$
\text{Similarity Score} = \begin{cases}
\frac{\text{Number of Overlapping CPC groups}}{\text{Total CPC groups in New Patent and Cited Patents}} & \text{if Total CPC groups} > 0 \\
NaN & \text{otherwise}
\end{cases}
$$

8. Calculate the Technological Leap Score = 
$$1 - \text{Similarity Score}$$

In [35]:
# 1. Create a dictionary mapping patent IDs to a list of patent IDs they cite
citations_list = citations.groupby("patent_id")["citation_patent_id"].apply(list)
citations_dict = citations_list.to_dict()

In [36]:
# 2. Create a dictionary mapping patent IDs to a list of their CPC groups
cpc_groups = cpc.groupby("patent_id")["cpc_group"].apply(set)
cpc_groups = pd.merge(
    cpc_groups,
    fwd_citations.set_index("patent_id"), 
    left_index=True, 
    right_index=True,
    how="right",
    validate="1:1"
)
cpc_groups = cpc_groups["cpc_group"]
cpc_groups = cpc_groups.rename("cpc_groups")
cpc_groups.head()

patent_id
10000000    {G01S7/4865, G01S17/894, G01S7/4917, G01S7/486...
10000001    {B29C2945/76083, B29C45/64, B29C45/7653, G05B2...
10000002    {B32B2475/00, B32B2307/554, B29D2030/0682, B32...
10000003    {B29C2049/4881, B29C49/20, B29L2031/7172, B29C...
10000004    {B29C48/9105, B29K2105/258, B29K2023/00, B29C4...
Name: cpc_groups, dtype: object

In [37]:
cpc_groups.isna().sum()

763040

In [38]:
cpc_groups_dict = cpc_groups.to_dict()

In [39]:
# 3. Get the intersection of CPC groups between a patent and its backward citations
def get_intersect_cpc_groups(row):
    patent_id = row.name
    patent_cpc_groups = cpc_groups_dict.get(patent_id, {})
    if not patent_cpc_groups or (isinstance(patent_cpc_groups, float) and np.isnan(patent_cpc_groups)):
        return []
    backward_citations = citations_dict.get(patent_id, [])
    if not backward_citations:
        return []
    backward_citations_cpc_groups = []
    for citation in backward_citations:
        citation_cpc_groups = cpc_groups_dict.get(citation, {})
        if not citation_cpc_groups or (isinstance(citation_cpc_groups, float) and np.isnan(citation_cpc_groups)):
            continue
        if citation_cpc_groups:
            backward_citations_cpc_groups.extend(citation_cpc_groups)
    if not backward_citations_cpc_groups:
        return []
    return list(set(patent_cpc_groups) & set(backward_citations_cpc_groups))

patent_cpc_subclass_pairs_with_atypicality["intersect_cpc_groups"] = patent_cpc_subclass_pairs_with_atypicality.apply(get_intersect_cpc_groups, axis=1)
patent_cpc_subclass_pairs_with_atypicality.head()

,cpc_subclass_pairs,atypicality_score,cpc_subclasses,intersect_cpc_groups
patent_id,,,,
10000000,[],NaN,{G01S},[]
10000001,"[(B29C, G05B)]",8.984207,"{G05B, B29C}","[B29C45/1751, B29C2945/76083, B29C45/76]"
10000002,"[(B29C, B29D), (B29C, B29K), (B29C, B29L), (B2...",8.375340,"{B32B, B29L, B29K, B60C, B29C, B29D}","[B32B27/08, B32B27/34, B29C48/21]"
10000003,"[(B29C, B29D), (B29C, B29K), (B29C, B29L), (B2...",6.990229,"{B29K, B29L, B29D, B29C}","[B29C49/20, B29L2031/7172, B29C2049/4881, B29C..."
10000004,"[(B29C, B29K), (B29C, B29L), (B29K, B29L)]",6.239670,"{B29K, B29L, B29C}",[B29C51/14]


In [40]:
# 4. Get the union of CPC groups between a patent and its backward citations
def get_union_cpc_groups(row):
    patent_id = row.name
    patent_cpc_groups = cpc_groups_dict.get(patent_id, {})
    if not patent_cpc_groups or (isinstance(patent_cpc_groups, float) and np.isnan(patent_cpc_groups)):
        return []
    backward_citations = citations_dict.get(patent_id, [])
    if not backward_citations:
        return patent_cpc_groups
    backward_citations_cpc_groups = []
    for citation in backward_citations:
        citation_cpc_groups = cpc_groups_dict.get(citation, {})
        if not citation_cpc_groups or (isinstance(citation_cpc_groups, float) and np.isnan(citation_cpc_groups)):
            continue
        if citation_cpc_groups:
            backward_citations_cpc_groups.extend(citation_cpc_groups)
    return list(set(patent_cpc_groups) | set(backward_citations_cpc_groups))

patent_cpc_subclass_pairs_with_atypicality["union_cpc_groups"] = patent_cpc_subclass_pairs_with_atypicality.apply(get_union_cpc_groups, axis=1)
patent_cpc_subclass_pairs_with_atypicality.head()

,cpc_subclass_pairs,atypicality_score,cpc_subclasses,intersect_cpc_groups,union_cpc_groups
patent_id,,,,,
10000000,[],NaN,{G01S},[],"[G01S7/4865, G01S17/89, Y02A90/10, G01S17/894,..."
10000001,"[(B29C, G05B)]",8.984207,"{G05B, B29C}","[B29C45/1751, B29C2945/76083, B29C45/76]","[G05B2219/45244, B29C45/83, B29C45/1751, B29C4..."
10000002,"[(B29C, B29D), (B29C, B29K), (B29C, B29L), (B2...",8.375340,"{B32B, B29L, B29K, B60C, B29C, B29D}","[B32B27/08, B32B27/34, B29C48/21]","[B32B2475/00, B32B2457/08, B32B2307/554, B29D2..."
10000003,"[(B29C, B29D), (B29C, B29K), (B29C, B29L), (B2...",6.990229,"{B29K, B29L, B29D, B29C}","[B29C49/20, B29L2031/7172, B29C2049/4881, B29C...","[B29C2791/006, B29C51/267, B29C49/0691, B29L20..."
10000004,"[(B29C, B29K), (B29C, B29L), (B29K, B29L)]",6.239670,"{B29K, B29L, B29C}",[B29C51/14],"[B32B2369/00, B65B31/04, B32B27/32, B65D75/580..."


In [41]:
# 5. Calculate the Technological Leap Score
# Technological Leap Score = 1 - (Jaccard Similarity)
def calculate_tech_leap_score(patent):
    # If a patent and all its backward citations have cite no CPC groups, it is incomparable.
    if not patent["union_cpc_groups"]:
        return np.nan
    # If a patent has no CPC subclasses in common with its backward citations, it has maximum technological leap.
    elif not patent["intersect_cpc_groups"]:
        return 1.0
    else:
        return 1 - (len(patent["intersect_cpc_groups"]) / len(patent["union_cpc_groups"]))
patent_cpc_subclass_pairs_with_atypicality["tech_leap"] = patent_cpc_subclass_pairs_with_atypicality.apply(calculate_tech_leap_score, axis=1)
patent_cpc_subclass_pairs_with_atypicality.head(10)

,cpc_subclass_pairs,atypicality_score,cpc_subclasses,intersect_cpc_groups,union_cpc_groups,tech_leap
patent_id,,,,,,
10000000,[],NaN,{G01S},[],"[G01S7/4865, G01S17/89, Y02A90/10, G01S17/894,...",1.000000
10000001,"[(B29C, G05B)]",8.984207,"{G05B, B29C}","[B29C45/1751, B29C2945/76083, B29C45/76]","[G05B2219/45244, B29C45/83, B29C45/1751, B29C4...",0.906250
10000002,"[(B29C, B29D), (B29C, B29K), (B29C, B29L), (B2...",8.375340,"{B32B, B29L, B29K, B60C, B29C, B29D}","[B32B27/08, B32B27/34, B29C48/21]","[B32B2475/00, B32B2457/08, B32B2307/554, B29D2...",0.949153
10000003,"[(B29C, B29D), (B29C, B29K), (B29C, B29L), (B2...",6.990229,"{B29K, B29L, B29D, B29C}","[B29C49/20, B29L2031/7172, B29C2049/4881, B29C...","[B29C2791/006, B29C51/267, B29C49/0691, B29L20...",0.818182
10000004,"[(B29C, B29K), (B29C, B29L), (B29K, B29L)]",6.239670,"{B29K, B29L, B29C}",[B29C51/14],"[B32B2369/00, B65B31/04, B32B27/32, B65D75/580...",0.980769
10000005,"[(B29C, Y10T)]",6.483157,"{Y10T, B29C}","[B29C51/082, B29C51/04, B29C51/262]","[B29C2791/006, Y10S425/053, B29C51/10, B29C51/...",0.750000
10000006,"[(B29C, B29K), (B29C, B29L), (B29C, B60R), (B2...",7.613158,"{B29L, B60R, B29K, B29C, Y10T}",[],"{B29C2791/006, B60R13/02, B29L2031/3041, B29K2...",1.000000
10000007,"[(B29C, B29K), (B29C, B29L), (B29K, B29L)]",6.239670,"{B29K, B29L, B29C}","[B29C57/04, B29K2023/0691, B29L2023/22]","[B26D5/12, Y10T29/49998, F15B2211/3059, F15B22...",0.968085
10000008,"[(A44C, B29C), (A44C, B29K), (A44C, B29L), (B2...",9.454074,"{B29K, B29L, A44C, B29C}",[],"[B29C33/405, B44C5/0446, Y10S264/55, Y10S425/0...",1.000000


In [42]:
# 6. Save the final DataFrame to a CSV file
patent_cpc_subclass_pairs_with_atypicality.to_csv("./data/patents_with_atypicality_and_tech_leap.csv")

In [43]:
patent_cpc_subclass_pairs_with_atypicality["tech_leap"].describe()

count    6.744779e+06
mean     9.048763e-01
std      1.475424e-01
min      0.000000e+00
25%      8.750000e-01
50%      9.507389e-01
75%      1.000000e+00
max      1.000000e+00
Name: tech_leap, dtype: float64

## Technological Leap Score summary statistics
| Technological Leap Score Statistic | Value     |
|-----------------------------------|-----------|
| Number of non-null values         | 6,744,779 |
| Number of null values             | 763,040 |
| Total Count                       | 7,507,819 |
| Mean                              | 0.9049   |
| Standard Deviation                | 0.1475    |
| Min                               | 0.0000    |
| Max                               | 1.0000    |
| 25th-percentile                   | 0.8750    |
| 50th-percentile                   | 0.9507    |
| 75th-percentile                   | 1    |

In [44]:
# 7. Validate that all the patents with NaN technological leap scores are indeed patents
# that cite no CPC groups and have no backward citations or its backward citations have no CPC groups
assert patent_cpc_subclass_pairs_with_atypicality[patent_cpc_subclass_pairs_with_atypicality["tech_leap"].isna()]["union_cpc_groups"].apply(lambda x: len(x) == 0).all(), "There are patents with NaN technological leap scores that have CPC groups."
print("All patents with NaN technological leap scores have no CPC groups.")

All patents with NaN technological leap scores have no CPC groups.


In [48]:
# 8. Validate that all technological leap scores are between 0 and 1
assert patent_cpc_subclass_pairs_with_atypicality["tech_leap"].dropna().between(0.0, 1.0, inclusive="both").all(), "There are technological leap scores outside the range [0, 1]."
print("All technological leap scores are between 0 and 1.")

All technological leap scores are between 0 and 1.


In [51]:
# 9. Random spot check: Validate that the technological leap score for a random patent matches the expected value
import random
def manual_tech_leap(patent_id):
    backward_citations = citations_dict.get(patent_id, [])
    patent_cpc_groups = cpc_groups_dict.get(patent_id, {})
    backward_citations_cpc_groups = []
    if backward_citations:
        for citation in backward_citations:
            citation_cpc_groups = cpc_groups_dict.get(citation, {})
            if not citation_cpc_groups or (isinstance(citation_cpc_groups, float) and np.isnan(citation_cpc_groups)):
                continue
            if citation_cpc_groups:
                backward_citations_cpc_groups.extend(citation_cpc_groups)
    if not patent_cpc_groups or (isinstance(patent_cpc_groups, float) and np.isnan(patent_cpc_groups)):
        return np.nan
    if not backward_citations_cpc_groups:
        return 1.0
    union_cpc_groups = list(set(patent_cpc_groups) | set(backward_citations_cpc_groups))
    intersect_cpc_groups = list(set(patent_cpc_groups) & set(backward_citations_cpc_groups))
    if not union_cpc_groups:
        return np.nan
    if not intersect_cpc_groups:
        return 1.0
    else:
        return 1 - (len(intersect_cpc_groups) / len(union_cpc_groups))

samples = random.sample(patent_cpc_subclass_pairs_with_atypicality.index.tolist(), 10000)
for sample in samples:
    expected_score = manual_tech_leap(sample)
    actual_score = patent_cpc_subclass_pairs_with_atypicality.loc[sample, "tech_leap"]
    assert np.isclose(expected_score, actual_score, equal_nan=True), f"Expected {expected_score}, but got {actual_score} for patent {sample}."
print("All manual checks for technological leap scores passed successfully.")

All manual checks for technological leap scores passed successfully.


In [52]:
# 10. Validate that the output dataset contains the expected number of patents
num_patents_in_output = patent_cpc_subclass_pairs_with_atypicality.shape[0]
assert num_patents_in_output == num_patents, f"Number of patents in the output dataset ({num_patents_in_output}) does not match the number of patents in the input dataset ({num_patents})."
print("Number of patents in the output dataset matches the number of patents in the input dataset.")

Number of patents in the output dataset matches the number of patents in the input dataset.
